What you need: Internet connection

### What is "API"?

### Prep

In [ ]:
import requests

from PIL import Image
from IPython.display import display

cat_base_url = "https://api.thecatapi.com/v1"

# IMPORTANT: In reality such key serves as your 'password' and should be only read from a separate file which is excluded from email attachments, file uploads and git commits etc.
# It should never be 'hardcoded' in your code.
cat_api_key = "live_J53mvL4PgbDmZHQLhdbJqOfFbLTYyxTbQ3TfgUomPwf5jDpVmRuoO2SDZcySbnX2"

ensembl_base_url = "https://rest.ensembl.org/"

def display_image(url):
    img = Image.open(requests.get(response[0]['url'], stream=True).raw)
    display(img)

### The Cat API

In [ ]:
# https://developers.thecatapi.com/view-account/ylX4blBYT9FaoVd6OhvR?report=aZyiLrsCh#tag/Images/paths/~1images~1search/get

r = requests.get(cat_base_url + "/images/search")
print('Request has been made to:', r.url)

response = r.json()
cat_url = response[0]['url']
cat_id = response[0]['id']

response

In [ ]:
r = requests.get(cat_base_url + "/images/" + cat_id + "/breeds")
print('Request has been made to:', r.url)
response = r.json()

response

### The Ensembl API

In [ ]:
# https://rest.ensembl.org/documentation/info/lookup

ensembl_id = 'ENSG00000157764'

r = requests.get(ensembl_base_url + 'lookup/id/' + ensembl_id, headers={
    "Content-Type" : "application/json"
})

r.json()

# Can we include phenotypes in the response?